In [256]:
import pandas as pd
import functools
import numpy as np

In [321]:
x

,First Name,Last Name,Display Name,Nickname,E-mail Address,E-mail 2 Address,E-mail 3 Address,Home Phone,Business Phone,Home Fax,...,Notes,Birthday,Anniversary,Gender,Web Page,Web Page 2,Categories,Unnamed: 37,Unnamed: 38,Unnamed: 39
4052,Beverly,Melugin,Beverly Melugin,NaN,Beverly.Melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4053,Beverly,Melugin,Beverly Melugin,NaN,beverly.melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4054,Beverly,Melugin,Beverly Melugin,NaN,beverly.melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4055,Beverly,Melugin,Beverly Melugin,NaN,Beverly.Melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4056,Beverly,Melugin,Beverly Melugin,NaN,Beverly.Melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4057,Beverly,Melugin,Beverly Melugin,NaN,beverly.melugin@doj.ca.gov,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [386]:
class Contacts:
    def __init__(self):
        self.Contacts = pd.read_csv("contacts.csv")
        self.fixNames()
        b = len(self.Contacts)
        self.Contacts = self.Contacts.drop_duplicates(ignore_index=True)
        dup = b - len(self.Contacts) 
        self.add()
        a = len(self.Contacts)
        print("before:",b, "after:",a, "# of duplicates:",dup, "# compressed:", b-a)
        i = self.Contacts.groupby(["First Name","Last Name"])["Home Phone"].count()
        print("# of different people (unique first and last names):", len(i))
        self.Contacts["Display Name"] = self.Contacts[["First Name","Last Name"]].apply(self.replaceDisplayName,axis=1)
    
    def replaceDisplayName(self,x):
        f = x["First Name"]
        l = x["Last Name"]
        if pd.isnull(f):
            return l
        if pd.isnull(l):
            return f
        return f + " " + l
    
    def splitname(self,name):
        n = name.split(" ")
        if len(n)==1:
            return [n[0],None]
        return [" ".join(n[:-1]), n[-1]]
    def fixNames(self):
        v = self.Contacts.loc[~self.Contacts["First Name"].isnull() & self.Contacts["First Name"].str.contains("'"),["First Name","Last Name"]].values.tolist()
        newNames = [self.splitname(first.strip("'")) if first[0]=="'" and first[-1]=="'" else [first,last] for first,last in v ]
        self.Contacts.loc[~self.Contacts["First Name"].isnull() & self.Contacts["First Name"].str.contains("'"),["First Name","Last Name"]] = newNames

    
    def merge(self,person):
        newperson = []
        for feature in person.columns:
            vals = list(person[feature].dropna().unique())
            if len(vals)==0:
                vals = [None]
            newF = functools.reduce(lambda a, b: str(a)+" "+str(b), vals)
            newperson.append(newF)
        return newperson
    def createNewPersonContacts(self, person):
        less_than_one_phone = len(person["Home Phone"].dropna().unique())<=1
        less_than_one_email = len(person["E-mail Address"].dropna().unique())<=1
        if less_than_one_phone and less_than_one_email:
            newperson = [self.merge(person)]
        elif not less_than_one_email and less_than_one_phone:
            newperson = person.groupby("E-mail Address").apply(self.merge).values
        elif not less_than_one_phone and less_than_one_email: 
            newperson = person.groupby("Home Phone").apply(self.merge).values
        else:
            newperson = person.groupby("E-mail Address").apply(self.merge).values
        newperson = list(newperson)
        return newperson
                
                
    def add(self):
        c = self.Contacts.groupby(["First Name","Last Name"],dropna=False)
        newC = []
        o = 0
        for i,person in c:
            newperson = self.createNewPersonContacts(person) 
            o += len(person)
            newC += newperson
        
        print(len(self.Contacts),o)
            
        self.Contacts = pd.DataFrame(newC,columns=self.Contacts.columns)
        self.Contacts = self.Contacts.sort_values(by=["First Name","Last Name"])

In [387]:
c = Contacts()
c.Contacts.to_csv("newContacts.csv",index=False)

4148 4148
before: 6771 after: 3729 # of duplicates: 2623 # compressed: 3042
# of different people (unique first and last names): 1956


In [388]:
c.Contacts.to_csv("newContacts.csv",index=False)

In [381]:
np.nan != np.nan

True

In [391]:
c.Contacts.iloc[20:24].to_csv("practice Contacts.csv",index=False)

1